# New component

Type here a description for this component

In [ ]:
%%writefile Model.py
import logging
from typing import List, Iterable, Dict, Union

import numpy as np
import pandas as pd
from platiagro import load_model

logger = logging.getLogger(__name__)


class Model(object):
    """
    Model template. You may load your model and artifacts in __init__ using platiagro SDK.

    Visit https://platiagro.github.io/sdk/ to find all the details about the SDK.
    """

    def __init__(self, dataset: str = None, target: str = None, experiment_id: str = None):
        logger.info(f"dataset: {dataset}")
        logger.info(f"target: {target}")
        logger.info(f"experiment_id: {experiment_id}")

        # Loads models and other artifacts using PlatIAgro SDK
        model = load_model(name=experiment_id)

        self.columns_names = ["col0", "col1", "col2", "col3"]

    def class_names(self):
        """The column names of the array that `predict` returns."""
        return self.columns_names

    def predict(self, X: np.ndarray, feature_names: Iterable[str], meta: Dict = None) -> Union[np.ndarray, List, str, bytes]:
        """Takes an array (numpy) X and feature_names and returns another array.

        Your return array should be at least 2-dimensional.

        Args:
            X (numpy.array): Array-like with data.
            feature_names (iterable, optional): Array of feature names.
            meta (dict, optional): Dict of metadata.
        """
        # Builds a DataFrame from numpy.array.
        df = pd.DataFrame(X, columns=feature_names)

        X_predict = df.to_numpy()

        # start your code here...

        return X_predict

## Deployment Test

It simulates a model deployed by PlatIAgro

In [ ]:
%%writefile env.sh
export MODEL_NAME="Model"
export API_TYPE="REST"
export SERVICE_TYPE="MODEL"
export PERSISTENCE=0
export LOG_LEVEL="DEBUG"
export PARAMETERS='[
{"type":"STRING","name":"dataset","value":"iris"},
{"type":"STRING","name":"target","value":"Species"},
{"type":"STRING","name":"experiment_id","value":"a71b85d0-6d92-4868-80a1-d2efd270ca5f"}]'

In [ ]:
%%bash
source env.sh
seldon-core-microservice "$MODEL_NAME" "$API_TYPE" \
    --service-type "$SERVICE_TYPE" \
    --persistence "$PERSISTENCE" \
    --parameters "$PARAMETERS" \
    --log-level "$LOG_LEVEL" > log.txt 2>&1 &

ATTEMPT=0
until $(curl --output /dev/null --silent --head --fail http://localhost:5000/health/ping); do
    # exit process if not healthy after 10 seconds
    if [ "$ATTEMPT" -gt 10 ]; then
        cat log.txt
        exit 1
    fi
    ATTEMPT=$((ATTEMPT + 1))
    sleep 1
done
echo "Deployment successful. Waiting for requests."

## Make predictions

In [ ]:
%%bash

curl -sSL localhost:5000/predict --data-binary @- << EOF
json={
    "data": {
        "names": ["SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"],
        "ndarray": [
            [5.1,3.5,1.4,0.2]
        ]
    },
    "meta": {}
}
EOF

## View logs

In [ ]:
!cat log.txt

## Clean up the test

In [ ]:
!ps -ef | grep [s]eldon-core-microservice | awk '{print $2}' | xargs -r kill